In [75]:
from huggingface_hub import notebook_login
notebook_login()

In [79]:
from datasets import Dataset, DatasetDict, Image
from pathlib import Path
import os
import numpy as np

In [107]:
image_folder_train = Path("data_crop_merged/train/image/")
res_label_folder_train = Path("data_crop_merged/train/res_label/")
shad_label_folder_train = Path("data_crop_merged/train/shad_label/")
image_paths_train = [str(file) for file in image_folder_train.iterdir() if file.is_file()]
res_label_paths_train = [str(file) for file in res_label_folder_train.iterdir() if file.is_file()]
shad_label_paths_train = [str(file) for file in shad_label_folder_train.iterdir() if file.is_file()]
image_paths_train.sort()
res_label_paths_train.sort()
shad_label_paths_train.sort()

In [108]:
assert len(image_paths_train) == len(res_label_paths_train) == len(shad_label_paths_train)

In [109]:
from PIL import Image as pil_image
for i in range(len(image_paths_train)):
    res_label_path = res_label_paths_train[i]
    shad_label_path = shad_label_paths_train[i]
    with pil_image.open(res_label_path) as res_mask_img:
        with pil_image.open(shad_label_path) as shad_mask_img:
            res_mask = np.array(res_mask_img)
            shad_mask = np.array(shad_mask_img)
    
            res_mask[res_mask == 255] = 1 # residue is 1 and soil is 0
            shad_mask[shad_mask == 255] = 1 # shaded is 1 and sunlit is 0
            new_mask = np.zeros_like(res_mask, dtype=np.uint8)
            
            # residue and sunlit
            new_mask[(res_mask == 1) & (shad_mask == 0)] = 1
            
            # residue and shaded
            new_mask[(res_mask == 1) & (shad_mask == 1)] = 2
            
            # background and sunlit
            new_mask[(res_mask == 0) & (shad_mask == 0)] = 3
            
            # background and shaded
            new_mask[(res_mask == 0) & (shad_mask == 1)] = 4
            # Create a new PIL image and save
            path_parts = res_label_path.split("/")
            path_parts[-1] = path_parts[-1].replace("res","res_shad")
            path_parts[-2] = path_parts[-2].replace("res_", "")
            res_file_path = "/".join(path_parts)
            path_parts = shad_label_path.split("/")
            path_parts[-1] = path_parts[-1].replace("sunshad","res_shad")
            path_parts[-2] = path_parts[-2].replace("shad_", "")
            shad_file_path = "/".join(path_parts)
            # print(res_file_path)
            # print(shad_file_path)
            assert res_file_path == shad_file_path
            mask_binary = pil_image.fromarray(new_mask)
            # file_path = res_label_path.name()
            mask_binary.save(res_file_path)


In [110]:
print(file_path)

data_crop_merged/train/label/IMG_0946_res_shad_part16.tif


In [111]:
label_folder_train = Path("data_crop_merged/train/label/")
label_paths_train = [str(file) for file in label_folder_train.iterdir() if file.is_file()]

In [112]:
# your images can of course have a different extension
# semantic segmentation maps are typically stored in the png format
# image_paths_train = ["path/to/image_1.jpg/jpg", "path/to/image_2.jpg/jpg", ..., "path/to/image_n.jpg/jpg"]
# label_paths_train = ["path/to/annotation_1.png", "path/to/annotation_2.png", ..., "path/to/annotation_n.png"]
print(label_paths_train)

['data_crop_merged/train/label/IMG_0629_res_shad_part01.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part02.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part03.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part04.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part05.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part06.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part07.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part08.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part09.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part10.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part11.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part12.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part13.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part14.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part15.tif', 'data_crop_merged/train/label/IMG_0629_res_shad_part16.tif', 'data_crop_merged/train

In [113]:
# same for validation
# image_paths_validation = [...]
# label_paths_validation = [...]

def create_dataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths),
                                "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())

    return dataset

In [114]:
# step 1: create Dataset objects
dataset = create_dataset(image_paths_train, label_paths_train)
# splits = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = splits["train"]
# validation_dataset = splits["test"] 

In [115]:



# step 2: create DatasetDict
dataset = DatasetDict({
    "train": dataset,
    # "validation": validation_dataset,
  }
)

In [117]:
# step 3: push to hub (assumes you have ran the huggingface-cli login command in a terminal/notebook)
dataset.push_to_hub("agaid_residue_and_shad")

# optionally, you can push to a private repo on the hub
# dataset.push_to_hub("name of repo on the hub", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/688 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Sowmith1999/agaid_residue_and_shad/commit/1a81a133642d80c6beb6602054a68af6178d679a', commit_message='Upload dataset', commit_description='', oid='1a81a133642d80c6beb6602054a68af6178d679a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Sowmith1999/agaid_residue_and_shad', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Sowmith1999/agaid_residue_and_shad'), pr_revision=None, pr_num=None)

In [101]:
print(shad_label_paths_train[27])

data_crop_merged/train/shad_label/IMG_0630_sunshad_part12.tif


In [116]:

example_mask_path = label_paths_train[45]
with pil_image.open(example_mask_path) as pil_new_mask:
    np_mask = np.array(pil_new_mask)
    print("Mask shape:", np_mask.shape)
    print(np_mask)
    np.savetxt("label_mask.csv", np_mask, delimiter=",", fmt="%d")
    print("Unique values:", np.unique(np_mask))
    # mask = np.array(img)
    # mask[mask == 255] = 1
    # # Create a new PIL image and save
    # mask_binary = PIL.Image.fromarray(mask)
    # mask_binary.save("./new_mask.tif")


Mask shape: (512, 512)
[[3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 [3 3 3 ... 3 3 3]
 ...
 [4 4 4 ... 3 3 3]
 [4 4 4 ... 3 3 3]
 [4 4 4 ... 3 3 3]]
Unique values: [1 2 3 4]


In [28]:
res_label_path = res_label_paths_train[0]
shad_label_path = shad_label_paths_train[0]
print(res_label_path)

data_crop_merged/train/res_label/IMG_0629_res_part01.tif


In [27]:
path_parts = res_label_path.split("/")
path_parts[-1] = path_parts[-1].replace("res","res_shad")
path_parts[-2] = path_parts[-2].replace("res_", "")
"/".join(path_parts)

'data_crop_merged/train/label/IMG_0629_res_shad_part01.tif'

In [29]:

with pil_image.open(res_label_path) as res_mask_img:
    with pil_image.open(shad_label_path) as shad_mask_img:
        res_mask = np.array(res_mask_img)
        shad_mask = np.array(shad_mask_img)

        res_mask[res_mask == 255] = 1 # residue is 1 and soil is 0
        shad_mask[shad_mask == 255] = 1 # shaded is 1 and sunlit is 0
        new_mask = np.zeros_like(res_mask, dtype=np.uint8)
        
        # residue and sunlit
        new_mask[(res_mask == 1) & (shad_mask == 0)] = 1
        
        # residue and shaded
        new_mask[(res_mask == 1) & (shad_mask == 1)] = 2
        
        # background and sunlit
        new_mask[(res_mask == 0) & (shad_mask == 0)] = 3
        
        # background and shaded
        new_mask[(res_mask == 0) & (shad_mask == 1)] = 4
        # Create a new PIL image and save
        path_parts = res_label_path.split("/")
        path_parts[-1] = path_parts[-1].replace("res","res_shad")
        path_parts[-2] = path_parts[-2].replace("res_", "")
        "/".join(path_parts)
        mask_binary = PIL.Image.fromarray(new_mask)
        # file_path = res_label_path.name()
        mask_binary.save("./merged_tif.tif")